In [ ]:
# get sample data
!mkdir sample
!wget https://oracle-livelabs.github.io/analytics-ai/oci-artificial-intelligence/ai-speech/object-storage/files/oracle-advertising.wav
!wget https://oracle-livelabs.github.io/analytics-ai/oci-artificial-intelligence/ai-speech/object-storage/files/oracle-redbull-racing.wav
!wget https://oracle-livelabs.github.io/analytics-ai/oci-artificial-intelligence/ai-speech/object-storage/files/ready-player-one.wav
!wget https://oracle-livelabs.github.io/analytics-ai/oci-artificial-intelligence/ai-speech/object-storage/files/the-poet.wav

translate wav file using ffmpeg

```bash
ffmpeg -i ./sample/oracle-advertising.wav -c copy -fflags +bitexact -flags:v +bitexact -flags:a +bitexact ./out/oracle-advertising.wav
ffmpeg -i ./sample/oracle-redbull-racing.wav -c copy -fflags +bitexact -flags:v +bitexact -flags:a +bitexact ./out/oracle-redbull-racing.wav
ffmpeg -i ./sample/ready-player-one.wav -c copy -fflags +bitexact -flags:v +bitexact -flags:a +bitexact ./out/ready-player.wav
ffmpeg -i ./sample/the-poet.wav -c copy -fflags +bitexact -flags:v +bitexact -flags:a +bitexact ./out/the-poet.wav
```

upload oci object storage

```bash
oci os bucket create --name speech-bucket -c $C
oci os object put --file ./out/oracle-advertising.wav --bucket-name speech-bucket
oci os object put --file ./out/oracle-redbull-racing.wav --bucket-name speech-bucket
oci os object put --file ./out/ready-player.wav --bucket-name speech-bucket
oci os object put --file ./out/the-poet.wav --bucket-name speech-bucket
```

In [ ]:
import oci
from oci.config import from_file

import os

ai_client = oci.ai_speech.AIServiceSpeechClient(oci.config.from_file())

# Give your job related details in these fields
DISPLAY_NAME = "speech-python-sdk-demo"
COMPARTMENT_ID = os.environ.get("C")
DESCRIPTION = "advertising"
NAMESPACE = "orasejapan"
BUCKET = "speech-bucket"
JOB_PREFIX = "python-sdk-demo"
LANGUAGE_CODE = "en-US"
FILE_NAMES = ["oracle-advertising.wav", "oracle-redbull-racing.wav", "ready-player.wav", "the-poet.wav"]
MODE_DETAILS = oci.ai_speech.models.TranscriptionModelDetails(domain="GENERIC", language_code=LANGUAGE_CODE)
OBJECT_LOCATION = oci.ai_speech.models.ObjectLocation(namespace_name=NAMESPACE, bucket_name=BUCKET,
object_names=FILE_NAMES)
INPUT_LOCATION = oci.ai_speech.models.ObjectListInlineInputLocation(
    location_type="OBJECT_LIST_INLINE_INPUT_LOCATION", object_locations=[OBJECT_LOCATION])
OUTPUT_LOCATION = oci.ai_speech.models.OutputLocation(namespace_name=NAMESPACE, bucket_name=BUCKET,
                                                             prefix=JOB_PREFIX)

transcription_job_details = oci.ai_speech.models.CreateTranscriptionJobDetails(
    display_name = DISPLAY_NAME,
    compartment_id = COMPARTMENT_ID,
    description = DESCRIPTION,
    model_details = MODE_DETAILS,
    input_location = INPUT_LOCATION,
    output_location = OUTPUT_LOCATION
)

transcription_job = None
print("*** CREATING TRANSCRIPTION JOB ***")

try:
    transcription_job = ai_client.create_transcription_job(
        create_transcription_job_details = transcription_job_details
    )
except Exception as e:
    print(e)
else:
    print(transcription_job.data)